In [1]:
# import requests

# url = "http://127.0.0.1:8000/predict"
# file_path = "/Users/kehindeelelu/Documents/aimechanics/dataset/equipment_sound_dataset/early_fault/early_fault_001.wav"

# with open(file_path, "rb") as f:
#     files = {"file": f}
#     response = requests.post(url, files=files)

# # Parse the JSON response
# response_data = response.json()

# # Access the predicted class
# print(response_data["prediction"]["predicted_class"])

early_fault


In [1]:
import subprocess
import os
import signal

def kill_port_8000():
    try:
        result = subprocess.run(
            ['lsof', '-i', ':8000'],
            stdout=subprocess.PIPE,
            stderr=subprocess.DEVNULL,
            text=True
        )

        lines = result.stdout.strip().split('\n')
        print(lines)
        
        if len(lines) <= 1:
            print("No process found using port 8000.")
            return

        for line in lines[1:]:  # Skip header
            parts = line.split()
            pid = int(parts[1])
            os.kill(pid, signal.SIGKILL)
            print(f"Killed process {pid} using port 8000.")
            
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    kill_port_8000()


['']
No process found using port 8000.


In [19]:
import requests
import sounddevice as sd
from scipy.io.wavfile import write
import tempfile
import time

# Define the server URL
url = "http://127.0.0.1:8000/predict"

# Define the folder to save audio files
audio_folder = "/Users/kehindeelelu/Documents/aimechanics/dataset/recorded_audio"

# Record audio from the microphone
def record_audio(duration=5, sample_rate=44100):
    print("Recording...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()  # Wait until recording is finished
    print("Recording complete.")
    return audio_data, sample_rate

# Save the recorded audio to a temporary WAV file
def save_audio_to_tempfile(audio_data, sample_rate):
    temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    write(temp_file.name, sample_rate, audio_data)
    return temp_file.name

# Save the recorded audio to a specific folder
def save_audio_to_folder(audio_data, sample_rate, folder, filename="recorded_audio.wav"):
    file_path = os.path.join(folder, filename)
    write(file_path, sample_rate, audio_data)
    print(f"Audio saved to {file_path}")
    return file_path

# Main logic
if __name__ == "__main__":
    # Record audio for 5 seconds
    audio_data, sample_rate = record_audio(duration=5)
    print("Audio data shape:", audio_data.shape)

    # Save the audio to the specified folder with a unique filename
    timestamp = int(time.time())
    filename = f"audio_{timestamp}.wav"
    audio_file_path = save_audio_to_folder(audio_data, sample_rate, audio_folder, filename)

    # Save the audio to a temporary file
    temp_file_path = save_audio_to_tempfile(audio_data, sample_rate)

    # Debug: Save the audio locally to verify recording
    write("test_audio.wav", sample_rate, audio_data)
    print("Audio saved as test_audio.wav")

    # Send the audio file to the server
    with open(audio_file_path, "rb") as f:
        files = {"file": f}
        # response = requests.post(url, files=files)
        response = requests.post(url, files=files, params={"timestamp": time.time()})
        
    # Parse the JSON response
    response_data = response.json()
    print("Server response:", response_data)

    # Access the predicted class
    print("Predicted class:", response_data.get("prediction", {}).get("predicted_class"))

    # Clean up temporary file
    import os
    os.remove(temp_file_path)

Recording...
Recording complete.
Audio data shape: (220500, 1)
Audio saved to /Users/kehindeelelu/Documents/aimechanics/dataset/recorded_audio/audio_1744735455.wav
Audio saved as test_audio.wav
Recording complete.
Audio data shape: (220500, 1)
Audio saved to /Users/kehindeelelu/Documents/aimechanics/dataset/recorded_audio/audio_1744735455.wav
Audio saved as test_audio.wav
Server response: {'prediction': {'predicted_class': 'failure', 'probabilities': [0.11489042777731871, 0.12375686969257803, 0.7613527025301032]}}
Predicted class: failure
Server response: {'prediction': {'predicted_class': 'failure', 'probabilities': [0.11489042777731871, 0.12375686969257803, 0.7613527025301032]}}
Predicted class: failure
